# Notebook, used to generate 3D plots and export them to html.

In [1]:
import pandas as pd
import numpy as np
import pyvista as pv

from os import listdir
from os.path import join

from src.tvData import get_df, get_cloud, get_nn_mask, mark_valid_signal, median_filter, get_dist
from src.constants import NN_THRESHOLD, FILES_S4, GOOD_FILES_S4

In [2]:
DATA_FOLDER = "./data/"
CMAP_2 = ["red", "yellow"]

In [3]:
def get_plotter(title: str) -> pv.Plotter:
    """Create Plotter object with title.

    Args:
        title (str): title

    Returns:
        pv.Plotter: pv plotter object
    """
    pl = pv.Plotter(notebook=True)
    pl.add_title(title)
    pl.enable_eye_dome_lighting()
    pl.show_grid()
    pl.show_axes()
    return pl


In [8]:
html_file_s4 = "htmls_dump/gr_singstrecke_s4_14_files.html"

df_s4 = get_df(FILES_S4, join(DATA_FOLDER, "gr_ringstrecke_s4/"))
cloud_s4 = get_cloud(df_s4)
pl_s4 = get_plotter("Grosse Ringstrecke s4")
pl_s4.add_points(
    cloud_s4,
    color="blue",
    render_points_as_spheres=True
)
pl_s4.export_html(html_file_s4)
pl_s4.show()


Widget(value="<iframe src='http://localhost:59776/index.html?ui=P_0x1be240070a0_0&reconnect=auto' style='width…

In [9]:
html_file_good_s4 = "htmls_dump/gr_singstrecke_s4_good_3_files.html"

df_good_s4 = get_df(GOOD_FILES_S4, join(DATA_FOLDER, "gr_ringstrecke_s4/"))
cloud_good_s4 = get_cloud(df_good_s4)
pl_s4_good = get_plotter("Grosse Ringstrecke s4")
pl_s4_good.add_points(
    cloud_good_s4,
    color="blue",
    render_points_as_spheres=True
)
pl_s4_good.export_html(html_file_good_s4)
pl_s4_good.show()

Widget(value="<iframe src='http://localhost:59776/index.html?ui=P_0x1be277853a0_1&reconnect=auto' style='width…

In [10]:
nn_threshold = 50
html_file_nn_s4_colored = f"htmls_dump/gr_singstrecke_s4_14_files_nn_{nn_threshold}_colored.html"
nn_mask_s4 = get_nn_mask(df_s4, nn_threshold)
cloud_s4_nn_colored = get_cloud(df_s4.iloc[1:-1])
cloud_s4_nn_colored["color_mask"] = nn_mask_s4
pl_s4_nn_colored = get_plotter(f"S4, NN {nn_threshold} Masked, yellow is valid, red is invalid")
pl_s4_nn_colored.add_points(
    cloud_s4_nn_colored,
    cmap=CMAP_2,
    scalars="color_mask"
)
pl_s4_nn_colored.export_html(html_file_nn_s4_colored)
pl_s4_nn_colored.show()


Widget(value="<iframe src='http://localhost:59776/index.html?ui=P_0x1be604cf250_2&reconnect=auto' style='width…

In [11]:
html_file_nn_s4 = f"htmls_dump/gr_singstrecke_s4_14_files_nn_{nn_threshold}.html"
df_nn = df_s4.iloc[1:-1].loc[nn_mask_s4]
cloud_s4_nn = get_cloud(df_nn)
pl_s4_nn = get_plotter(f"S4, NN {nn_threshold} Masked")
pl_s4_nn.add_points(
    cloud_s4_nn,
    color="blue"
)
pl_s4_nn.export_html(html_file_nn_s4)
pl_s4_nn.show()

Widget(value="<iframe src='http://localhost:59776/index.html?ui=P_0x1be3b7fbb20_3&reconnect=auto' style='width…

In [18]:
html_file_run2 = "htmls_dump/run2_master_blo.html"
folder_run2 = join(DATA_FOLDER, "run2/")
df_run2 = get_df(listdir(folder_run2), folder_run2)
cloud_run2 = get_cloud(df_run2)
pl_run2 = get_plotter("Run 2, Dachsburgring, Master in BLO")
pl_run2.add_points(
    cloud_run2,
    color="blue",
    render_points_as_spheres=True
)
pl_run2.export_html(html_file_run2)
pl_run2.show()

Widget(value="<iframe src='http://localhost:59776/index.html?ui=P_0x1be24462e50_23&reconnect=auto' style='widt…

In [13]:
html_file_run8 = "htmls_dump/run8_master_ger.html"

folder_run8 = join(DATA_FOLDER, "run8/")
df_run8 = get_df(listdir(folder_run8), folder_run8)
cloud_run8 = get_cloud(df_run8)
pl_run8 = get_plotter("Run 8, Dachsburgring, Master in BLO")
pl_run8.add_points(
    cloud_run8,
    color="blue"
)
pl_run8.export_html(html_file_run8)
pl_run8.show()

Widget(value="<iframe src='http://localhost:59776/index.html?ui=P_0x1be23c2ec40_5&reconnect=auto' style='width…

In [4]:
df_recv, recv_flg = mark_valid_signal(
    get_df(FILES_S4, join(DATA_FOLDER, "gr_ringstrecke_s4/")),
    min_distance=3000,
    max_distance=12000
)
cloud_recv = get_cloud(df_recv)
for i in range(recv_flg.shape[1]):
    cloud_recv["color_mask"] = recv_flg[:, i]
    pl = get_plotter(f"Receiver Nr. {i}")
    pl.add_points(
        cloud_recv,
        cmap=CMAP_2,
        scalars="color_mask"
    )
    pl.export_html(f"htmls_dump/recv_colored/gr_ringstrecke_s4_recv_{i}.html")
cloud_recv["color_mask"] = df_recv["got_signal_flg"]
pl_recv = get_plotter(f"Receiver Nr. {i}")
pl_recv.add_points(
    cloud_recv,
    cmap=CMAP_2,
    scalars="color_mask"
)
pl_recv.export_html(f"htmls_dump/gr_ringstrecke_s4_satelliten_abdeckung.html")

In [16]:
kernel_size = 29
html_file_med_s4 = f"htmls_dump/med_filter_k{kernel_size}.html"
coords = np.c_[df_nn.x, df_nn.y, df_nn.z]

cloud_med_s4 = pv.PolyData(median_filter(coords, kernel_size=kernel_size))
pl_s4 = get_plotter(f"Medianfilter Kernelgrösse: {kernel_size}")
pl_s4.add_points(
    cloud_med_s4,
    color="blue",
    render_points_as_spheres=True
)
pl_s4.export_html(html_file_med_s4)
pl_s4.show()

c:\Users\Habbasch\envs\ip6\lib\site-packages\pyvista\core\utilities\points.py:52: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value="<iframe src='http://localhost:59776/index.html?ui=P_0x1be25bf52e0_21&reconnect=auto' style='widt…